# Más desarrollos de modelos, empleando más funcionalidades de Scikit-Learn
> Dataset disponible en https://www.kaggle.com/datasets/ziya07/college-student-management-dataset

Este dataset incluye múltiples características relacionadas con el rendimiento estudiantil, métricas de uso de plataformas de aprendizaje, donde la variable objetivo es el riesgo de abandono. Comencemos con la acción.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("./datasets/college_student_management_data.csv")
df.head()

In [ ]:
df.info()

# EDA

Las variables están completas, por lo que no se requiere de imputación por valores faltantes. A continuación, se visualizará la distribución de las variables.

In [ ]:
df.drop(columns=['student_id'], inplace=True)

In [ ]:
numerical_columns = df.select_dtypes("number").columns
numerical_columns

In [ ]:
categorical_columns = df.select_dtypes("object").columns
categorical_columns

In [ ]:
# Histograms for each numerical column
df[numerical_columns].hist(figsize=(18, 12), bins=20, layout=(2, 5))
plt.suptitle('Histograms of Numerical Columns', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# Box plots for each numerical column
fig, axes = plt.subplots(2, 5, figsize=(18, 10))
for ax, col in zip(axes.flatten(), numerical_columns):
    df.boxplot(column=col, ax=ax)
    ax.set_title(col)
plt.suptitle('Box Plots of Numerical Columns', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

No se presentan outliers univariados en ninguna de las variables encontradas. Las variables tampoco parecen seguir una distribución estadística clara como la normal.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
for ax, col in zip(axes.flatten(), categorical_columns):
    df[col].value_counts().plot(kind='bar', ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.set_title(col)
    ax.set_ylabel('Counts')

La clase `risk_level` está claramente desbalanceada. Se usará creación de datos sintéticos para balancear las clases.

# Preprocesamiento
Se convertirá la variable objetivo a una numérica ordinal mediante LabelEncoder, y las demás variables predictoras serán transformadas mediante OneHotEncoder.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

categorical_columns = categorical_columns.drop(['risk_level'])

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
le = LabelEncoder()

# Se elimina la columna 'risk_level' de X y se aplica OneHotEncoder a las demás columnas categóricas, manteniendo la estructura del dataframe.
X = df.copy().drop(columns=['risk_level'])
X.drop(columns=categorical_columns, inplace=True)
X = pd.concat([X, pd.DataFrame(ohe.fit_transform(df[categorical_columns]), columns=ohe.get_feature_names_out(categorical_columns))], axis=1)
X.head()

y = le.fit_transform(df['risk_level'])

In [ ]:
display(pd.DataFrame(X).head())
display(y)

A continuación, a partir de X, se crearán dos versiones: Una normalizada y otra escalada.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

std = StandardScaler()
mm = MinMaxScaler()

X_std = X.copy()
X_std[numerical_columns] = std.fit_transform(X[numerical_columns])
X_mm = pd.DataFrame(mm.fit_transform(X), columns=X.columns)

In [ ]:
display(X_std.head())
display(X_mm.head())

# Sobre la detección de outliers multivariados

Este es un campo que aún no exploro, y la idea es aprender y probar metodologías.

De buenas a primeras se me ocurre que mediante técnicas de aprendizaje no supervisado como K-means se pueden encontrar outliers, donde estos se encontrarían a una distancia grande de los centroides. No obstante, también hay otros métodos de aprendizaje no supervisado, como DBSCAN, que es de clustering, o IsolationForest. 

**DBSCAN** consiste en identificar **grupos de puntos que estén muy cerca entre sí** (o regiones con alta densidad), marcando como ruido o outliers las regiones que están poco denamente pobladas, con el beneficio de que los datos **no tienen que estar linealmente separados**; ni siquiera que tengan una forma específica [ver imagen de ejemplo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/DBSCAN-density-data.svg/1024px-DBSCAN-density-data.svg.png). En contraparte, asume que los grupos a encontrar tienen la misma cantidad de observaciones, y puede tener problemas al encontrar clusters con densidades diferentes, [para lo cual fue creado HDBSCAN](https://scikit-learn.org/stable/modules/clustering.html#hdbscan).

**Isolation Forest** es un algoritmo basado en árboles de decisión. Asumiendo que los **outliers son pocos y diferentes de la muestra**, se construyen múltiples **árboles seleccionando aleatoriamente una característica**, y eligiendo un umbral de acuerdo con el dominio de aplicación. El valor de corte sirve para dividir los datos y generar un mínimo y máximo, los cuales serán divididos recursivamente hasta (1.) **aislar cada instancia en una hoja del árbol**, o (2.) **alcanzar una profundidad máxima**. De esta manera, los outliers requerirán de menos hojas para quedar aislados, siendo esta la técnica para identificarlos.

---

En este ejercicio, me interesa evaluar el rendimiento de la creación de múltiples modelos empleando DBSCAN, IsolationForest, o sin aplicar ningún tipo de detección de outliers, con el fin de evaluar cómo se ve afectado el entrenamiento por estos datos. Para ello, se utilizarán múltiples *Pipelines* de Scikit-Learn, donde la configuración de los modelos de machine learning para realizar la clasificación serán comunes entre todos los pipelines, y lo que variará será el tratamiento de preprocesamiento dado a los datos.

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.7, min_samples=2).fit(X_mm, y)
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Aparentemente, DBSCAN detecta todos los datos como ruido bajo estos ajustes. En una primera instancia, me sugiere que todos los datos están homogéneamente distribuidos, pero no tendría sentido, dado que implicaría que no hay forma de distinguir las clases entre sí. Otra interpretación es que la distancia promedio entre los puntos es mayor a 0.7, para lo cual un análisis mediante el método del codo puede ser revelador. Se usará KNN con $n = 5$ para graficar las distancias al 4to vecino, y elegir un valor de inflexión.

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt

neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(X_mm)
dists, _ = neigh.kneighbors(X_mm)
k_dist = np.sort(dists[:, -1])  # distancia al 4.º vecino
plt.plot(k_dist)
plt.ylabel("4th nearest neighbor distance")
plt.xlabel("Points sorted by distance")
plt.show()

Los valores _adecuados_ estarían en el intervalo $\text{eps} \in [1.1, ~1.2]$, donde se produce el codo superior. Probando con la cota inferior:

In [ ]:
dbscan = DBSCAN(eps=1.1, min_samples=5).fit(X_mm, y)
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Y con la cota superior:

In [ ]:
dbscan = DBSCAN(eps=1.2, min_samples=5).fit(X_mm, y)
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

En ningún caso la cantida de outliers es elevada, donde para la cota inferior el porcentaje de puntos ruidosos es menor al 1.4%.

Como parte de lo mencionado anteriormente, resulta interesante comparar estos resultados con IsolationForest. No obstante, no sé muy bien cómo hacer esto. Sin ajustar hiperparámetros todas las observaciones son categorizadas como outliers. Tampoco conozco de ninguna heurística o método para ajustar el parámetro `contamination`, así que lo único que se me ocurre es usar el porcentaje de outliers detectados en el paso anterior.

Dado el porcentaje de outliers detectado por DBSCAN a raíz del valor `eps` hallado de manera empírica por el método de KNN, tomo la decisión de no eliminarlos. Son un porcentaje bastante ínfimo que no considero que pueda ensuciar los modelos.

In [ ]:
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(contamination=0.015, random_state=42)
iso_forest.fit(X_mm)
outliers = iso_forest.predict(X_mm)
outliers = np.where(outliers == -1, 1, 0)  # Convert -1 to 1 (outlier) and 1 to 0 (inlier)
# Count the number of outliers
n_outliers = np.sum(outliers)
print(f"Number of outliers detected: {n_outliers}")

# Balanceo de clases

Principalmente, `imbalanced-learning` implementa métodos métodos estadísticos y basados en machine learning. En este ejercicio, me interesa realizar over-sampling de las clases de la variable objetivo con menos observaciones (séase `medium` y `low`). De los métodos estadísticos destacan SMOTE y ADASYN, las cuales interpolan los datos ya existentes para generar nuevas muestras. 

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_mm, y)

In [ ]:
display(pd.DataFrame(X_resampled).info())
display(y_resampled)

# Selección de características

De nuevo, me basaré en las metodologías disponibles en scikit-learn. Como métodos univariados, se pueden seleccionar las K características con mejor puntaje, o aquellas características que superen un percentil de puntaje. Este función de puntaje se deja a elección, y para este notebook será `mutual_info_classif`. 

In [ ]:
from sklearn.feature_selection import SelectKBest,  SelectPercentile,mutual_info_classif

kbest = SelectKBest(score_func=mutual_info_classif, k=7).fit(X_resampled, y_resampled)
X_kbest = kbest.transform(X_resampled)
print("Selected features based on mutual information:")
selected_features = X_kbest.shape[1]
print(f"Number of selected features: {selected_features}")
selected_feature_names = X.columns[kbest.get_support()]
print(f"Selected feature names: {selected_feature_names.tolist()}")

In [ ]:
top70 = SelectPercentile(score_func=mutual_info_classif, percentile=70).fit(X_resampled, y_resampled)
X_top70 = top70.transform(X_resampled)
print("Selected features based on 70th percentile of mutual information:")
selected_features_70 = X_top70.shape[1]
print(f"Number of selected features: {selected_features_70}")
selected_feature_names_70 = X.columns[top70.get_support()]
print(f"Selected feature names: {selected_feature_names_70.tolist()}")

Los resultados comparten algunas características en común. En todo caso, es posible explicar el dataset con menos variables de las originales, al menos desde una perspectiva univariada. Las perspectivas multivariadas, según he leído, se basan en análisis de rendimiento de modelos a medida de que se quitan características. Para entrenar modelos ya estoy yo, así que me quedaré con las top 7 variables y con eso entrenaré algunos modelos.

# Entrenamiento

Me gustan los modelos populares, así que iré con perceptron multicapas, máquinas de soporte vectorial, procesos gaussianos, y quizás alguno como bagging o boosting.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_grid = [
    {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (25, 25, 25)],
        'activation': ['relu', 'logistic'],
        'alpha': np.linspace(0.0001, 0.01, 5),
        'learning_rate': ['constant', 'adaptive'],
        'solver': ['adam'],
        'max_iter': [200]
    }
]

mlp_cv = GridSearchCV(MLPClassifier(random_state=42), mlp_grid, scoring='f1_weighted', cv=5, n_jobs=-1, verbose=2)
mlp_cv.fit(X_train, y_train)

In [ ]:
display(f"Best estimator: {mlp_cv.best_estimator_}")
display(f"Best score: {mlp_cv.best_score_}")

In [ ]:
mlp_best_estimator = mlp_cv.best_estimator_
y_pred = mlp_best_estimator.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

probando por algún motivo con tensorflow

In [ ]:
from sklearn.svm import SVC
svm_grid = [
    {
        'C': np.logspace(-3, 3, 7),
        'kernel': ['linear'],
        'max_iter': [1000]
    },
    {
        'C': np.logspace(-3, 3, 7),
        'kernel': ['rbf'],
        'gamma': ['scale', 'auto'],
        'max_iter': [1000]
    },
    {
        'C': np.logspace(-3, 3, 7),
        'kernel': ['poly'],
        'degree': [2, 3, 4, 5, 6],
        'gamma': ['scale', 'auto'],
        'max_iter': [1000]
    }

]

svm_cv = GridSearchCV(SVC(random_state=42), svm_grid, scoring='f1_weighted', cv=5, n_jobs=-1, verbose=2)
svm_cv.fit(X_train, y_train)

In [ ]:
display(f"Best estimator: {svm_cv.best_estimator_}")
display(f"Best score: {svm_cv.best_score_}")

In [ ]:
svm_best_estimator = svm_cv.best_estimator_
y_pred = svm_best_estimator.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb_grid = [
    {
        'max_iter': [200],
        'max_depth': [3, 5, 7],
        'learning_rate': np.linspace(0.0001, 0.1, 5),
        'l2_regularization': [0.0, 0.1, 0.2],
        'max_leaf_nodes': [None, 10, 20],
        'min_samples_leaf': [1, 5, 10],
        'max_bins': [255],
    }
]
hgb_cv = GridSearchCV(HistGradientBoostingClassifier(random_state=42), hgb_grid, scoring='f1_weighted', cv=5, n_jobs=-1, verbose=2)
hgb_cv.fit(X_train, y_train)

In [ ]:
display(f"Best estimator: {hgb_cv.best_estimator_}")
display(f"Best score: {hgb_cv.best_score_}")

In [ ]:
svm_best_estimator = svm_cv.best_estimator_
y_pred = svm_best_estimator.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))